In [1]:
!nvidia-smi

Tue Dec 26 03:15:30 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  On   | 00000001:00:00.0 Off |                    0 |
| N/A   46C    P0   130W / 400W |  46140MiB / 81920MiB |    100%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA A100-SXM...  On   | 00000002:00:00.0 Off |                    0 |
| N/A   

In [2]:
# !pip3 install scipy bitsandbytes --user

In [3]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [4]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch
import json

TORCH_DTYPE = 'bfloat16'
nf4_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=getattr(torch, TORCH_DTYPE)
)

In [5]:
from transformers.trainer_utils import get_last_checkpoint

latest = get_last_checkpoint("fpf-1b-instructions-16k")
latest

'fpf-1b-instructions-16k/checkpoint-5500'

In [6]:
tokenizer = AutoTokenizer.from_pretrained(latest)

In [7]:
model = AutoModelForCausalLM.from_pretrained(
    latest,
    use_flash_attention_2 = True,
    quantization_config = nf4_config
)

In [8]:
# model = AutoModelForCausalLM.from_pretrained(
#     latest,
#     use_flash_attention_2 = True,
#     torch_dtype = torch.bfloat16,
# )
# _ = model.cuda()

In [9]:
def parse_mistral_chat(messages, function_call = None):

    user_query = messages[-1]['content']

    users, assistants = [], []
    for q in messages[:-1]:
        if q['role'] == 'user':
            users.append(q['content'])
        elif q['role'] == 'assistant':
            assistants.append(q['content'])

    texts = ['<s>']
    
    if function_call:
        fs = []
        for f in function_call:
            f = json.dumps(f, indent=4)
            fs.append(f)
        fs = '\n\n'.join(fs)
        texts.append(f'\n[FUNCTIONCALL]\n{fs}\n')
        
    for u, a in zip(users, assistants):
        texts.append(f'[INST] {u.strip()} [/INST] {a.strip()}</s>')

    texts.append(f'[INST] {user_query.strip()} [/INST]')
    prompt = ''.join(texts).strip()
    return prompt

In [19]:
s = """
hey gila
"""

messages = [
    {'role': 'user', 'content': s.strip()},
]
prompt = parse_mistral_chat(messages)
inputs = tokenizer([prompt], return_tensors='pt', add_special_tokens=False).to('cuda')
generate_kwargs = dict(
    inputs,
    max_new_tokens=128,
    top_p=0.95,
    top_k=50,
    temperature=0.9,
    do_sample=True,
    num_beams=1,
)
r = model.generate(**generate_kwargs)
print(tokenizer.decode(r[0]))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> [INST] hey gila [/INST] hello, saya di sini untuk membantu! Saya boleh memberikan maklumat tentang pelbagai topik atau menjawab soalan anda tentang pelbagai subjek.</s>


In [10]:
s = """
objektif: Bahagian yang di dalam kurungan dalam ayat-ayat yang berikut mungkin mengandungi kesalahan bahasa dan mungkin juga tidak. Tandakan sama ada A, B atau C jika ayat itu mengandungi kesalahan dan tandakan D jika tiada kesalahan.
soalan: Halimah percaya (percaya benar) anaknya tidak terbabit dengan tuduhan jenayah kolar putih.
A. sungguh percaya benar
B. teramat percaya benar
C. sangat percaya benar
D. percaya benar
"""

messages = [
    {'role': 'user', 'content': s.strip()}
]
prompt = parse_mistral_chat(messages)
inputs = tokenizer([prompt], return_tensors='pt', add_special_tokens=False).to('cuda')
generate_kwargs = dict(
    inputs,
    max_new_tokens=128,
    top_p=0.95,
    top_k=50,
    temperature=0.9,
    do_sample=True,
    num_beams=1,
)
r = model.generate(**generate_kwargs)
print(tokenizer.decode(r[0]))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> [INST] objektif: Bahagian yang di dalam kurungan dalam ayat-ayat yang berikut mungkin mengandungi kesalahan bahasa dan mungkin juga tidak. Tandakan sama ada A, B atau C jika ayat itu mengandungi kesalahan dan tandakan D jika tiada kesalahan.
soalan: Halimah percaya (percaya benar) anaknya tidak terbabit dengan tuduhan jenayah kolar putih.
A. sungguh percaya benar
B. teramat percaya benar
C. sangat percaya benar
D. percaya benar [/INST] D</s>


In [11]:
messages = [
    {'role': 'user', 'content': 'tolong tukar 10 ringgit ke usd'}
]
prompt = parse_mistral_chat(messages)
inputs = tokenizer([prompt], return_tensors='pt', add_special_tokens=False).to('cuda')
generate_kwargs = dict(
    inputs,
    max_new_tokens=128,
    top_p=0.95,
    top_k=50,
    temperature=0.9,
    do_sample=True,
    num_beams=1,
)
r = model.generate(**generate_kwargs)
print(tokenizer.decode(r[0]))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> [INST] tolong tukar 10 ringgit ke usd [/INST] Untuk menukar 10 ringgit kepada dolar, langkah pertama ialah memasukkannya ke dalam dolar terdekat. Sebagai contoh, jika anda ingin menukar 10 ringgit kepada dolar, anda akan memasukkannya ke dalam dolar terdekat. Jika anda ingin menukar 10 dolar kepada mata wang lain, anda perlu menukarnya dahulu. Jika anda memerlukan contoh yang lebih terperinci, beritahu saya dan saya akan cuba membantu anda dengan lebih baik.</s>


In [12]:
c = """
knowledge base: Getting started Introduction Quickstart TypeScript License Changelog Integration guides React Next.js Vue Nuxt Svelte Angular NEW Astro Remix Meteor.js NEW Gatsby SolidJS Qwik Laravel NEW Symfony Rails Phoenix Django Flask Customize Configuration Dark mode Theming Colors Icons Optimization Components Accordion Alerts Avatar Badge Banner Bottom Navigation Breadcrumb Buttons Button Group Card Carousel Device Mockups Drawer Dropdowns Footer Forms Gallery Indicators Jumbotron KBD List Group Mega menu Modal Navbar Pagination Popover Progress Rating Sidebar Skeleton Speed Dial Spinner Stepper Tables Tabs Timeline Toast Tooltips Typography Video Forms Input Field File Input Search Input Select Textarea Checkbox Radio Toggle Range Floating Label Typography Headings Paragraphs Blockquote Images Lists Links Text HR Plugins Charts NEW Datepicker
user: apa knowledge base awak?
bot:Pangkalan pengetahuan saya termasuk pengenalan, panduan pantas, lesen, perubahan log, dan panduan penyepaduan dengan pelbagai rangka seperti React, Next.js, Vue, Nuxt, Svelte, Angular, Astro, Remix, Meteor.js, Gatsby, SolidJS, Qwik, Laravel, Symfony, Rails, Phoenix, Django, Flask, dan lain-lain.

Selain itu, ia juga merangkumi panduan untuk mengubah suai konfigurasi, menggunakan tema gelap, menetapkan warna, menggunakan ikon, mengoptimumkan komponen, dan pelbagai komponen seperti akordion, alert, avatar, badge, banner, navigasi bawah, serbuk roti, butang, kad, karusel, mockup peranti, laci, dropdown, footer, form, galeri, penunjuk, jumbotron, KBD, senarai kumpulan, mega menu, modal, navbar, pagination, popover, kemajuan, penarafan, sidebar, rangka, speed dial, spinner, stepper, tables, tabs, timeline, toast, tooltips, typography, video, dan banyak lagi.

Terdapat juga pemalam seperti carta dan datepicker dalam pangkalan pengetahuan saya. Jika terdapat soalan yang lebih khusus tentang sesuatu dalam pangkalan pengetahuan ini, sila beritahu saya. Saya sedia membantu.

Explain step-by-step very details with relationships why `bot` respond like that to `user` given on `knowledge base` input?
"""
messages = [
    {'role': 'user', 'content': c.strip()}
]
prompt = parse_mistral_chat(messages)
inputs = tokenizer([prompt], return_tensors='pt', add_special_tokens=False).to('cuda')
generate_kwargs = dict(
    inputs,
    max_new_tokens=128,
    top_p=0.95,
    top_k=50,
    temperature=0.9,
    do_sample=True,
    num_beams=1,
)
r = model.generate(**generate_kwargs)
print(tokenizer.decode(r[0]))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> [INST] knowledge base: Getting started Introduction Quickstart TypeScript License Changelog Integration guides React Next.js Vue Nuxt Svelte Angular NEW Astro Remix Meteor.js NEW Gatsby SolidJS Qwik Laravel NEW Symfony Rails Phoenix Django Flask Customize Configuration Dark mode Theming Colors Icons Optimization Components Accordion Alerts Avatar Badge Banner Bottom Navigation Breadcrumb Buttons Button Group Card Carousel Device Mockups Drawer Dropdowns Footer Forms Gallery Indicators Jumbotron KBD List Group Mega menu Modal Navbar Pagination Popover Progress Rating Sidebar Skeleton Speed Dial Spinner Stepper Tables Tabs Timeline Toast Tooltips Typography Video Forms Input Field File Input Search Input Select Textarea Checkbox Radio Toggle Range Floating Label Typography Headings Paragraphs Blockquote Images Lists Links Text HR Plugins Charts NEW Datepicker
user: apa knowledge base awak?
bot:Pangkalan pengetahuan saya termasuk pengenalan, panduan pantas, lesen, perubahan log, dan p

In [13]:
s = """
CREATE TABLE pelajar (
   id SERIAL PRIMARY KEY,
   name VARCHAR(50),
   email VARCHAR(50),
   student_number VARCHAR(50),
   course VARCHAR(50),
   start_date DATE,
   end_date DATE,
   grade VARCHAR(50)
)

berdasarkan table definition, bina sql query untuk filter pelajar yang grade C keatas
"""

messages = [
    {'role': 'user', 'content': s.strip()}
]
prompt = parse_mistral_chat(messages)
inputs = tokenizer([prompt], return_tensors='pt', add_special_tokens=False).to('cuda')
generate_kwargs = dict(
    inputs,
    max_new_tokens=256,
    top_p=0.95,
    top_k=50,
    temperature=0.9,
    do_sample=True,
    num_beams=1,
)
r = model.generate(**generate_kwargs)
print(tokenizer.decode(r[0]))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> [INST] CREATE TABLE pelajar (
   id SERIAL PRIMARY KEY,
   name VARCHAR(50),
   email VARCHAR(50),
   student_number VARCHAR(50),
   course VARCHAR(50),
   start_date DATE,
   end_date DATE,
   grade VARCHAR(50)
)

berdasarkan table definition, bina sql query untuk filter pelajar yang grade C keatas [/INST] Anda boleh menggunakan kueri berikut untuk mendapatkan gred pelajar C ke atas dalam jadual "pelajar":

```sql
SELECT gred FROM pelajar ORDER BY gred DESC LIMIT 1;
```

Untuk membina kueri SQL yang lebih kompleks:

```sql
SELECT gred FROM pelajar
ORDER BY gred DESC
LIMIT 1;
```

Kueri SQL yang lengkap akan mengembalikan gred pelajar C ke atas, di mana gred adalah nilai gred dari jadual "pelajar" dalam jadual "pelajar". Dalam kueri di atas, jika tiada pelajar yang memenuhi kriteria, fungsi `ORDER BY` akan mengembalikan gred secara langsung. Untuk mendapatkan gred pelajar C ke atas, kueri pertama akan menggunakan klausa `ORDER BY` dan kemudian menggunakan klausa `LIMIT` untuk menge

In [14]:
messages = [
    {'role': 'user', 'content': 'create simple postgres table untuk student'}
]
prompt = parse_mistral_chat(messages)
inputs = tokenizer([prompt], return_tensors='pt', add_special_tokens=False).to('cuda')
generate_kwargs = dict(
    inputs,
    max_new_tokens=256,
    top_p=0.95,
    top_k=50,
    temperature=0.9,
    do_sample=True,
    num_beams=1,
)
r = model.generate(**generate_kwargs)
print(tokenizer.decode(r[0]))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> [INST] create simple postgres table untuk student [/INST] Untuk mencipta jadual PostgreSQL yang ringkas untuk pelajar dalam bahasa Melayu, ikuti langkah-langkah berikut:

1. Buka teks editor anda.

2. Letakkan fail "postgres.sql" sebagai destinasi.

3. Kemudian, buka fail yang baru anda cipta.

4. Masukkan butiran yang diperlukan.

5. Setelah berjaya membuat sambungan, tekan 'Fail sambungan baru telah dibuat'.

6. Buka pangkalan data PostgreSQL anda, dan klik "Mula" atau "Pilih pangkalan data".

7. Di bahagian bawah jadual "pelajar" (biasanya dalam fail "nama_fail.sql"), anda akan menemui satu lajur yang mungkin anda ingin paparkan.

8. Klik "Jalankan" untuk menjalankan sambungan.

9. Sambungan anda kini akan dibuka dalam pangkalan data.

Itu sahaja, anda telah mencipta jadual untuk pelajar anda dengan berjaya.

Pastikan untuk memeriksa dokumentasi sambungan pangkalan data anda untuk maklumat lanjut.</s>


In [15]:
messages = [
    {'role': 'user', 'content': 'contoh 2d convolution guna numpy'}
]
prompt = parse_mistral_chat(messages)
inputs = tokenizer([prompt], return_tensors='pt', add_special_tokens=False).to('cuda')
generate_kwargs = dict(
    inputs,
    max_new_tokens=256,
    top_p=0.95,
    top_k=50,
    temperature=0.9,
    do_sample=True,
    num_beams=1,
)
r = model.generate(**generate_kwargs)
print(tokenizer.decode(r[0]))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> [INST] contoh 2d convolution guna numpy [/INST] Anda boleh menggunakan NumPy untuk menghasilkan matriks rawak menggunakan operasi "conv" pada matriks dan kemudian menggunakan `np.zeros` untuk mengisi matriks dengan nilai 0 pada setiap elemennya. Berikut adalah contoh kod:

```python
import numpy as np
import matplotlib.pyplot as plt

# Matriks dengan nilai 0 pada setiap elemennya
mat_np = np.zeros((2, 4))

# Menambahkannya ke plot
plt.plot([1, 2, 3, 4], [1, 1, 2, 3])

# Menetapkan semula plot
plt.show()
```

Dalam contoh ini, kami menggunakan `np.zeros` untuk mengisi matriks rawak dengan nilai 0 pada setiap elemennya. Kemudian, dengan `plt.plot`, plot akan dicipta dengan `plt.show`.

Anda boleh menukar matriks dengan nilai lain seperti dalam contoh yang diberikan.</s>


In [16]:
messages = [
    {'role': 'user', 'content': 'kwsp tu apa'}
]
prompt = parse_mistral_chat(messages)
inputs = tokenizer([prompt], return_tensors='pt', add_special_tokens=False).to('cuda')
generate_kwargs = dict(
    inputs,
    max_new_tokens=256,
    top_p=0.95,
    top_k=50,
    temperature=0.9,
    do_sample=True,
    num_beams=1,
)
r = model.generate(**generate_kwargs)
print(tokenizer.decode(r[0]))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> [INST] kwsp tu apa [/INST] KWSP merujuk kepada Kumpulan Wang Simpanan Pekerja.</s>


In [17]:
messages = [
    {'role': 'user', 'content': 'kenapa maybank leceh sangat'}   
]
prompt = parse_mistral_chat(messages)
inputs = tokenizer([prompt], return_tensors='pt', add_special_tokens=False).to('cuda')
generate_kwargs = dict(
    inputs,
    max_new_tokens=256,
    top_p=0.95,
    top_k=50,
    temperature=0.9,
    do_sample=True,
    num_beams=1,
)
r = model.generate(**generate_kwargs)
print(tokenizer.decode(r[0]))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> [INST] kenapa maybank leceh sangat [/INST] Maybank memang menghadapi cabaran dengan peraturan dan garis panduan yang kompleks, tetapi ia boleh menjadi mudah bagi sesetengah orang. Walau bagaimanapun, terdapat beberapa perkara yang perlu diperbaiki untuk bank, seperti:

1. Perubahan dalam waktu operasi: Maybank mungkin memperkenalkan beberapa perubahan dalam waktu operasi bank, seperti mengurangkan waktu buka atau menghentikan operasi, atau memperkenalkan waktu perbankan tambahan.

2. Kekurangan maklumat: Maybank mungkin memerlukan maklumat tertentu semasa membuka akaun atau melakukan transaksi, yang boleh menyebabkan pelanggan yang tidak layak perlu mendapatkan penjelasan lanjut atau penjelasan mengenai perubahan ini.

3. Perkhidmatan pelanggan: Maybank perlu terus meningkatkan perkhidmatan pelanggan mereka dan memastikan terdapat saluran komunikasi yang baik antara pelanggan dan kaunter. Ini boleh membantu untuk mengelakkan kekecewaan pelanggan.

4. Penempatan fungsi: Maybank mung

In [18]:
messages = [
    {'role': 'user', 'content': 'kenapa najib razak korup'}   
]
prompt = parse_mistral_chat(messages)
inputs = tokenizer([prompt], return_tensors='pt', add_special_tokens=False).to('cuda')
generate_kwargs = dict(
    inputs,
    max_new_tokens=256,
    top_p=0.95,
    top_k=50,
    temperature=0.9,
    do_sample=True,
    num_beams=1,
)
r = model.generate(**generate_kwargs)
print(tokenizer.decode(r[0]))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> [INST] kenapa najib razak korup [/INST] Najib Razak dianggap sebagai salah satu perompak terburuk dalam sejarah Malaysia kerana peranannya yang didakwa terlibat dalam skandal 1Malaysia Development Berhad (1MDB). Dia didakwa telah bersekongkol dan memanipulasi wang negara Malaysia untuk kepentingan peribadinya, yang kemudiannya mengakibatkan kerugian berbilion dolar.

Berikut adalah beberapa sebab mengapa Najib Razak telah dikaitkan dengan skandal 1MDB dan tuduhan rasuahnya:

1. **Penggunaan Dana Awam**: Najib Razak dituduh menggunakan dana awam untuk kepentingan peribadinya dengan cara yang paling membimbangkan. Beliau dituduh menggunakan dana yang berasal dari 1MDB untuk membiayai parti politiknya, Pertubuhan Kebangsaan Melayu Bersatu (UMNO), dan untuk memberi sogokan kepada orang ramai.

2. **Menyembunyikan Rasuah**: Skandal 1MDB dilihat sebagai salah satu kes salah laku kewangan terbesar dalam sejarah. Najib Razak dituduh gagal memberikan jawapan yang memuaskan kepada soalan men

In [19]:
messages = [
    {'role': 'user', 'content': '1+1 / 10'}   
]
prompt = parse_mistral_chat(messages)
inputs = tokenizer([prompt], return_tensors='pt', add_special_tokens=False).to('cuda')
generate_kwargs = dict(
    inputs,
    max_new_tokens=256,
    top_p=0.95,
    top_k=50,
    temperature=0.9,
    do_sample=True,
    num_beams=1,
)
r = model.generate(**generate_kwargs)
print(tokenizer.decode(r[0]))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> [INST] 1+1 / 10 [/INST] 1+1 = 1
10 = 10
Oleh itu, jawapannya ialah 1</s>


In [24]:
messages = [
    {'role': 'user', 'content': 'babi boleh terbang tak'}   
]
prompt = parse_mistral_chat(messages)
inputs = tokenizer([prompt], return_tensors='pt', add_special_tokens=False).to('cuda')
generate_kwargs = dict(
    inputs,
    max_new_tokens=256,
    top_p=0.95,
    top_k=50,
    temperature=0.9,
    do_sample=True,
    num_beams=1,
)
r = model.generate(**generate_kwargs)
print(tokenizer.decode(r[0]))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> [INST] babi boleh terbang tak [/INST] Saya tidak makan daging babi. Anda boleh makan, tetapi tidak boleh terbang.</s>


In [25]:
messages = [
    {'role': 'user', 'content': 'apa tu kata hubung'}   
]
prompt = parse_mistral_chat(messages)
inputs = tokenizer([prompt], return_tensors='pt', add_special_tokens=False).to('cuda')
generate_kwargs = dict(
    inputs,
    max_new_tokens=256,
    top_p=0.95,
    top_k=50,
    temperature=0.9,
    do_sample=True,
    num_beams=1,
)
r = model.generate(**generate_kwargs)
print(tokenizer.decode(r[0]))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> [INST] apa tu kata hubung [/INST] Kata hubung ialah perkataan yang digunakan untuk menyambungkan dua klausa atau ayat. Ia boleh jadi "apa", "di mana", "bilakah", "kenapa", "mengapa", dan sebagainya. Contoh: Dia datang ke pasar.</s>
